In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
X =pd.read_csv('train_ctrUa4K.csv', error_bad_lines=False)

X.head(20)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [3]:
Xt=pd.read_csv('test.csv', error_bad_lines=False)
#Xt.isnull().sum()

In [4]:
#targets = X.Loan_Status
#X.drop('Loan_Status', 1, inplace=True)

In [5]:
X.corr(method ='pearson') 
X.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [6]:
X['Gender'].fillna(X['Gender'].mode()[0], inplace=True)
X['Married'].fillna(X['Married'].mode()[0], inplace=True)
X['Self_Employed'].fillna(X['Self_Employed'].mode()[0], inplace=True)
X['Dependents'].fillna(X['Dependents'].mode()[0], inplace=True)
X['Loan_Amount_Term'].fillna(X['Loan_Amount_Term'].mode()[0], inplace=True)
X['Credit_History'].fillna(2, inplace=True)
X.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
X['Very_Short_Term'] = X['Loan_Amount_Term'].map(lambda t: 1 if t<=60 else 0)
X['Short_Term'] = X['Loan_Amount_Term'].map(lambda t: 1 if t>60 and t<180 else 0)
X['Long_Term'] = X['Loan_Amount_Term'].map(lambda t: 1 if t>=180 and t<=300  else 0)
X['Very_Long_Term'] = X['Loan_Amount_Term'].map(lambda t: 1 if t>300 else 0)
# X.drop('Loan_Amount_Term', axis=1, inplace=True)

X['Credit_History_Bad'] = X['Credit_History'].map(lambda c: 1 if c==0 else 0)
X['Credit_History_Good'] = X['Credit_History'].map(lambda c: 1 if c==1 else 0)
X['Credit_History_Unknown'] = X['Credit_History'].map(lambda c: 1 if c==2 else 0)
X.drop('Credit_History', axis=1, inplace=True)

In [8]:
X['Self_Employed'].value_counts()

No     532
Yes     82
Name: Self_Employed, dtype: int64

In [9]:
X['TotalIncome'] = X['ApplicantIncome'] + X['CoapplicantIncome']
X['TotalIncome_log'] = np.log(X['TotalIncome'])

# Xt['TotalIncome'] = Xt['ApplicantIncome'] + Xt['CoapplicantIncome']
# Xt['TotalIncome_log'] = np.log(Xt['TotalIncome'])
# Xt.drop(['ApplicantIncome','CoapplicantIncome'],axis=1)
X['total_income_by_loanAmt'] = X['TotalIncome'] / X['LoanAmount']
X['ApplicantIncome_by_loanAmt'] = X['ApplicantIncome'] / X['LoanAmount']
X['Loan_Amount_Term_By_amt'] = X['LoanAmount'] / X['Loan_Amount_Term']
X.drop(['ApplicantIncome','CoapplicantIncome'],axis=1)



,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Property_Area,Loan_Status,Very_Short_Term,Short_Term,Long_Term,Very_Long_Term,Credit_History_Bad,Credit_History_Good,Credit_History_Unknown,TotalIncome,TotalIncome_log,total_income_by_loanAmt,ApplicantIncome_by_loanAmt,Loan_Amount_Term_By_amt
0,LP001002,Male,No,0,Graduate,No,NaN,360.0,Urban,Y,0,0,0,1,0,1,0,5849.0,8.674026,NaN,NaN,NaN
1,LP001003,Male,Yes,1,Graduate,No,128.0,360.0,Rural,N,0,0,0,1,0,1,0,6091.0,8.714568,47.585938,35.804688,0.355556
2,LP001005,Male,Yes,0,Graduate,Yes,66.0,360.0,Urban,Y,0,0,0,1,0,1,0,3000.0,8.006368,45.454545,45.454545,0.183333
3,LP001006,Male,Yes,0,Not Graduate,No,120.0,360.0,Urban,Y,0,0,0,1,0,1,0,4941.0,8.505323,41.175000,21.525000,0.333333
4,LP001008,Male,No,0,Graduate,No,141.0,360.0,Urban,Y,0,0,0,1,0,1,0,6000.0,8.699515,42.553191,42.553191,0.391667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,71.0,360.0,Rural,Y,0,0,0,1,0,1,0,2900.0,7.972466,40.845070,40.845070,0.197222
610,LP002979,Male,Yes,3+,Graduate,No,40.0,180.0,Rural,Y,0,0,1,0,0,1,0,4106.0,8.320205,102.650000,102.650000,0.222222
611,LP002983,Male,Yes,1,Graduate,No,253.0,360.0,Urban,Y,0,0,0,1,0,1,0,8312.0,9.025456,32.853755,31.905138,0.702778
612,LP002984,Male,Yes,2,Graduate,No,187.0,360.0,Urban,Y,0,0,0,1,0,1,0,7583.0,8.933664,40.550802,40.550802,0.519444


In [10]:
# table = X.pivot_table(values='LoanAmount', index='Self_Employed' ,columns='Education', aggfunc=np.median)
# # Define function to return value of this pivot_table
# def fage(x):
#  return table.loc[x['Self_Employed'],x['Education']]
# # Replace missing values
# X['LoanAmount'].fillna(X[X['LoanAmount'].isnull()].apply(fage, axis=1), inplace=True)
# X['LoanAmount_log'] = np.log(X['LoanAmount'])
# X=X.drop(['ApplicantIncome','CoapplicantIncome'],axis=1)
# X['LoanAmount_log'].hist(bins=20)
# # X['LoanAmount'].fillna(X['LoanAmount'].mean(), inplace=True)

In [11]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']
le = LabelEncoder()
for i in var_mod:
    X[i] = le.fit_transform(X[i])
X.head()
X.corr(method ='pearson') 


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area,Loan_Status,Very_Short_Term,Short_Term,Long_Term,Very_Long_Term,Credit_History_Bad,Credit_History_Good,Credit_History_Unknown,TotalIncome,TotalIncome_log,total_income_by_loanAmt,ApplicantIncome_by_loanAmt,Loan_Amount_Term_By_amt
Gender,1.000000,0.364569,0.172914,0.045364,-0.000525,0.058809,0.082912,0.109520,-0.074030,-0.025752,0.017987,-0.004127,0.010999,0.086383,-0.082298,-0.009170,0.026655,-0.028981,0.093191,0.167285,-0.012072,-0.075325,0.061873
Married,0.364569,1.000000,0.334216,0.012304,0.004489,0.051708,0.075948,0.149743,-0.100912,0.004257,0.091478,-0.048180,0.046036,0.104793,-0.098572,-0.010938,-0.018150,0.041852,0.083319,0.151527,-0.049570,-0.088677,0.094022
Dependents,0.172914,0.334216,1.000000,0.055752,0.056798,0.118202,0.030430,0.166106,-0.103864,-0.000244,0.010118,0.005003,0.042427,0.105772,-0.113061,0.040160,-0.017523,-0.024881,0.125590,0.112812,-0.018260,0.006894,0.103853
Education,0.045364,0.012304,0.055752,1.000000,-0.010383,-0.140760,-0.062290,-0.171133,-0.073928,-0.065243,-0.085884,0.039872,-0.019599,0.088149,-0.086111,0.073658,-0.081637,0.030102,-0.161362,-0.204861,-0.081786,-0.070398,-0.081170
Self_Employed,-0.000525,0.004489,0.056798,-0.010383,1.000000,0.127180,-0.016100,0.117421,-0.033739,-0.030860,-0.003700,0.017700,0.048037,0.013660,-0.033295,0.001550,-0.004994,0.005645,0.113000,0.176709,0.106383,0.142661,0.054007
ApplicantIncome,0.058809,0.051708,0.118202,-0.140760,0.127180,1.000000,-0.116605,0.570909,-0.046531,-0.009500,-0.004710,-0.016294,-0.032502,0.093881,-0.071580,0.018615,0.006986,-0.034651,0.893037,0.717829,0.403511,0.542404,0.324292
CoapplicantIncome,0.082912,0.075948,0.030430,-0.062290,-0.016100,-0.116605,1.000000,0.188619,-0.059383,0.010522,-0.059187,0.002800,-0.030897,0.080482,-0.065025,-0.011134,-0.058795,0.104297,0.342781,0.383827,0.284357,-0.208191,0.135536
LoanAmount,0.109520,0.149743,0.166106,-0.171133,0.117421,0.570909,0.188619,1.000000,0.036981,-0.045792,-0.037318,-0.022893,-0.078374,0.030970,0.004212,0.001463,-0.035156,0.051531,0.624621,0.700809,-0.139872,-0.123010,0.491795
Loan_Amount_Term,-0.074030,-0.100912,-0.103864,-0.073928,-0.033739,-0.046531,-0.059383,0.036981,1.000000,-0.076120,-0.022549,-0.424520,-0.405325,-0.685661,0.884518,0.004705,0.002361,-0.009668,-0.070917,-0.056044,-0.107173,-0.087206,-0.501511
Property_Area,-0.025752,0.004257,-0.000244,-0.065243,-0.030860,-0.009500,0.010522,-0.045792,-0.076120,1.000000,0.032112,0.064780,-0.005112,0.074144,-0.084835,-0.001963,-0.018761,0.031234,-0.004218,-0.046246,0.074412,0.064084,0.038011


In [12]:
from sklearn.impute import KNNImputer
X_=X.copy()
X_.drop(['Loan_ID'],axis=1, inplace=True)
imputer = KNNImputer(n_neighbors=15)
imputed = imputer.fit_transform(X_)
ximp = pd.DataFrame(imputed, columns=X_.columns)
ximp.head()

# X['LoanAmount'].fillna(X['LoanAmount'].mean(), inplace=True)
# ximp=X.copy()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area,Loan_Status,Very_Short_Term,Short_Term,Long_Term,Very_Long_Term,Credit_History_Bad,Credit_History_Good,Credit_History_Unknown,TotalIncome,TotalIncome_log,total_income_by_loanAmt,ApplicantIncome_by_loanAmt,Loan_Amount_Term_By_amt
0,1.0,0.0,0.0,0.0,0.0,5849.0,0.0,155.266667,360.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5849.0,8.674026,39.546616,39.546616,0.450278
1,1.0,1.0,1.0,0.0,0.0,4583.0,1508.0,128.000000,360.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,6091.0,8.714568,47.585938,35.804688,0.355556
2,1.0,1.0,0.0,0.0,1.0,3000.0,0.0,66.000000,360.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3000.0,8.006368,45.454545,45.454545,0.183333
3,1.0,1.0,0.0,1.0,0.0,2583.0,2358.0,120.000000,360.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4941.0,8.505323,41.175000,21.525000,0.333333
4,1.0,0.0,0.0,0.0,0.0,6000.0,0.0,141.000000,360.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,6000.0,8.699515,42.553191,42.553191,0.391667


In [13]:
xs=ximp.copy()
target=xs['Loan_Status']
xs=xs.drop('Loan_Status',axis=1)
xs.head()


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area,Very_Short_Term,Short_Term,Long_Term,Very_Long_Term,Credit_History_Bad,Credit_History_Good,Credit_History_Unknown,TotalIncome,TotalIncome_log,total_income_by_loanAmt,ApplicantIncome_by_loanAmt,Loan_Amount_Term_By_amt
0,1.0,0.0,0.0,0.0,0.0,5849.0,0.0,155.266667,360.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5849.0,8.674026,39.546616,39.546616,0.450278
1,1.0,1.0,1.0,0.0,0.0,4583.0,1508.0,128.000000,360.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,6091.0,8.714568,47.585938,35.804688,0.355556
2,1.0,1.0,0.0,0.0,1.0,3000.0,0.0,66.000000,360.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3000.0,8.006368,45.454545,45.454545,0.183333
3,1.0,1.0,0.0,1.0,0.0,2583.0,2358.0,120.000000,360.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4941.0,8.505323,41.175000,21.525000,0.333333
4,1.0,0.0,0.0,0.0,0.0,6000.0,0.0,141.000000,360.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,6000.0,8.699515,42.553191,42.553191,0.391667


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xs, target, test_size=0.2, random_state=40)


In [15]:
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(n_estimators=200)
# model.fit(X_train, y_train)
# pred=model.predict(X_test)
# accuracy_score(y_test, pred)




In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
parameters ={"learning_rate"    : [ 0.1,0.20,0.30 ] ,
             "max_depth"        : [ 6,7 ],
             #"min_child_weight" : [ 1, 3, 5, 7 ],
             "gamma"            : [ 0.0],
             #"colsample_bytree" : [ 0.5,1],
             "random_state"     : [1,2,3,4]}
classifier = XGBClassifier(n_estimators=30)
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=parameters,
                           scoring='accuracy',
                           cv=10,
                           n_jobs=1)
model5= grid_search.fit(xs, target)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [17]:
from xgboost import XGBClassifier
classifier= XGBClassifier(n_estimators=30, max_depth=6,learning_rate=0.1,gamma=0,random_state=1)
classifier.fit(X_train,y_train)
print(best_parameters)
print(best_accuracy)
y_pred=model5.predict(X_test)
from sklearn import metrics
metrics.accuracy_score(y_pred, y_test)

{'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 6, 'random_state': 1}
0.799709148598625


0.8943089430894309

In [18]:
# pip install catboost

In [19]:
# from catboost import CatBoostRegressor
# from catboost import CatBoostClassifier, Pool
# model=CatBoostRegressor(iterations=100, depth=5, learning_rate=0.05, loss_function='RMSE')
# dtrain = Pool(data=X_train, label=y_train)
# model.fit(dtrain)
# pred=model.predict(X_test)
# accuracy_score(y_test, pred.round())

In [20]:
Xt.isnull().sum()
Xt['Gender'].fillna(Xt['Gender'].mode()[0], inplace=True)
Xt['Married'].fillna(Xt['Married'].mode()[0], inplace=True)
Xt['Self_Employed'].fillna(Xt['Self_Employed'].mode()[0], inplace=True)
Xt['Dependents'].fillna(Xt['Dependents'].mode()[0], inplace=True)
Xt['Loan_Amount_Term'].fillna(Xt['Loan_Amount_Term'].mode()[0], inplace=True)
Xt['Credit_History'].fillna(2, inplace=True)


Xt['Very_Short_Term'] = Xt['Loan_Amount_Term'].map(lambda t: 1 if t<=60 else 0)
Xt['Short_Term'] = Xt['Loan_Amount_Term'].map(lambda t: 1 if t>60 and t<180 else 0)
Xt['Long_Term'] = Xt['Loan_Amount_Term'].map(lambda t: 1 if t>=180 and t<=300  else 0)
Xt['Very_Long_Term'] = Xt['Loan_Amount_Term'].map(lambda t: 1 if t>300 else 0)
# Xt.drop('Loan_Amount_Term', axis=1, inplace=True)

Xt['Credit_History_Bad'] = Xt['Credit_History'].map(lambda c: 1 if c==0 else 0)
Xt['Credit_History_Good'] = Xt['Credit_History'].map(lambda c: 1 if c==1 else 0)
Xt['Credit_History_Unknown'] = Xt['Credit_History'].map(lambda c: 1 if c==2 else 0)
Xt.drop('Credit_History', axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder
var_mo = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area']
lee = LabelEncoder()
for i in var_mo:
    Xt[i] = lee.fit_transform(Xt[i])


In [21]:
from sklearn.impute import KNNImputer
Xt_=Xt.copy()
Xt_.drop(['Loan_ID'],axis=1, inplace=True)
imput = KNNImputer(n_neighbors=15)
imputd = imput.fit_transform(Xt_)
xtimp = pd.DataFrame(imputd, columns=Xt_.columns)
xtimp.head()
Xt=xtimp.copy()
Xt.head()
# Xt['LoanAmount'].fillna(Xt['LoanAmount'].mean(), inplace=True)
# Xt.drop(['Loan_ID'],axis=1, inplace=True)

Xt['TotalIncome'] = Xt['ApplicantIncome'] + Xt['CoapplicantIncome']
Xt['TotalIncome_log'] = np.log(Xt['TotalIncome'])
Xt.drop(['ApplicantIncome','CoapplicantIncome'],axis=1)
Xt['total_income_by_loanAmt'] = Xt['TotalIncome'] / Xt['LoanAmount']
Xt['ApplicantIncome_by_loanAmt'] = Xt['ApplicantIncome'] / Xt['LoanAmount']
Xt['Loan_Amount_Term_By_amt'] = Xt['LoanAmount'] / Xt['Loan_Amount_Term']
Xt.drop(['ApplicantIncome','CoapplicantIncome'],axis=1)


,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Property_Area,Very_Short_Term,Short_Term,Long_Term,Very_Long_Term,Credit_History_Bad,Credit_History_Good,Credit_History_Unknown,TotalIncome,TotalIncome_log,total_income_by_loanAmt,ApplicantIncome_by_loanAmt,Loan_Amount_Term_By_amt
0,1.0,1.0,0.0,0.0,0.0,110.0,360.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5720.0,8.651724,52.000000,52.000000,0.305556
1,1.0,1.0,1.0,0.0,0.0,126.0,360.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4576.0,8.428581,36.317460,24.412698,0.350000
2,1.0,1.0,2.0,0.0,0.0,208.0,360.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,6800.0,8.824678,32.692308,24.038462,0.577778
3,1.0,1.0,2.0,0.0,0.0,100.0,360.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4886.0,8.494129,48.860000,23.400000,0.277778
4,1.0,0.0,0.0,1.0,0.0,78.0,360.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3276.0,8.094378,42.000000,42.000000,0.216667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,1.0,1.0,3.0,1.0,1.0,113.0,360.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5786.0,8.663196,51.203540,35.477876,0.313889
363,1.0,1.0,0.0,0.0,0.0,115.0,360.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4867.0,8.490233,42.321739,36.156522,0.319444
364,1.0,0.0,0.0,0.0,0.0,126.0,360.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,5243.0,8.564649,41.611111,25.793651,0.350000
365,1.0,1.0,0.0,0.0,0.0,158.0,360.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,7393.0,8.908289,46.791139,31.645570,0.438889


In [22]:
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import RandomForestClassifier
# model1 = RandomForestClassifier(n_estimators=200)
# # model.fit(X_train, y_train)
# # pred=model.predict(X_test)
# # accuracy_score(y_test, pred)
# model1.fit(xs, target)
# pred1=model.predict(Xt)


# from catboost import CatBoostRegressor
# from catboost import CatBoostClassifier, Pool
# model=CatBoostRegressor(iterations=100, depth=5, learning_rate=0.05, loss_function='RMSE')
# dtrain = Pool(data=xs, label=target)
# model.fit(dtrain)
# pred=model.predict(Xt)



In [23]:
from xgboost import XGBClassifier
classifier= XGBClassifier(n_estimators=30, max_depth=6,learning_rate=0.1,gamma=0,random_state=1)
classifier.fit(xs,target)

pred1y=model5.predict(Xt)


In [24]:
df_output = pd.DataFrame()
aux = pd.read_csv('test.csv')
df_output['Loan_ID'] = aux['Loan_ID']
df_output['Loan_Status'] = np.vectorize(lambda s: 'Y' if s==1 else 'N')(pred1y)
df_output[['Loan_ID','Loan_Status']].to_csv('ou11xg.csv',index=False)